## Producer 1
Writing a python program that loads all the data from climate_streaming.csv and randomly (with replacement) feed the data to thestream every 10 seconds. 

In [1]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
import pandas as pd
import random
import json

In [ ]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{"10.192.96.162"}:9092'],
                                  api_version=(0, 10))

    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_str = str(json.dumps(data, sort_keys=True, indent=4))
        value_bytes = bytes(value_str, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

if __name__ == '__main__':
   
    topic = 'partB1'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    
    # read data from csv
    data = pd.read_csv('climate_streaming.csv')
    
    # Initialize date simulation
    latest_date = dt.datetime.strptime('2023-12-27', '%Y-%m-%d')
    
    array = []
    
        # Send data
    for index, row in data.iterrows():
        # Convert the row to a dictionary and add extra data
        message = row.to_dict()
        message['latitude'] = round(float(row['latitude']), 3)    # 3 decimal places
        message['longitude'] = round(float(row['longitude']), 3)  # 3 decimal places
        message['air_temperature_celcius'] = float(row['air_temperature_celcius'])
        message['relative_humidity'] = float(row['relative_humidity'])
        message['windspeed_knots'] = float(row['windspeed_knots'])
        message['max_wind_speed'] = float(row['max_wind_speed'])
        message['precipitation_value'] = float((row['precipitation '])[0:-1])   # numeric value from precipitation
        message['precipitation_type'] = row['precipitation '][-1]         # percipitation type from string 
        message['GHI_w/m2'] = (row['GHI_w/m2'])    
        message['created_date'] = (latest_date + dt.timedelta(days=index+1)).strftime('%Y-%m-%d')
        message['producer'] = 'producer1'

        array.append(message)

    for _ in range(len(array)):
        rand_num = random.randrange(0, len(array))
        random_data = array[rand_num]
        #key = 'producer1' which to be identified from consumers
        publish_message(producer, topic, "producer1", random_data)
        sleep(10)
    

Publishing records..
Message published successfully. Data: {'latitude': -36.152, 'longitude': 143.578, 'air_temperature_celcius': 15.0, 'relative_humidity': 56.1, 'windspeed_knots': 5.1, 'max_wind_speed': 9.9, 'precipitation ': ' 0.00I', 'GHI_w/m2': 122, 'precipitation_value': 0.0, 'precipitation_type': 'I', 'created_date': '2024-04-13', 'producer': 'producer1'}
Message published successfully. Data: {'latitude': -36.551, 'longitude': 142.579, 'air_temperature_celcius': 17.0, 'relative_humidity': 51.2, 'windspeed_knots': 9.9, 'max_wind_speed': 18.1, 'precipitation ': ' 0.00G', 'GHI_w/m2': 144, 'precipitation_value': 0.0, 'precipitation_type': 'G', 'created_date': '2024-12-15', 'producer': 'producer1'}
Message published successfully. Data: {'latitude': -37.434, 'longitude': 148.123, 'air_temperature_celcius': 14.0, 'relative_humidity': 43.7, 'windspeed_knots': 17.0, 'max_wind_speed': 27.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 126, 'precipitation_value': 0.0, 'precipitation_type': 'I',